In [104]:
import sys
import os
import glob
import pandas as pd

In [105]:
def read_files_into_dataframe(file):
    data = []
    with open(file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            columns = line.strip().split()
            if len(columns) <= 2:
                data.append(columns)
    df = pd.DataFrame(data, columns=["filename", "solvability_score"])  # Add column names
    return df

In [106]:
# Read files into dataframes
directory = "/mnt/atlas_local/guantong/home/New_manuscript/subset_patient_cases_by_difficulty/difficulty_subsets"
file_pattern = "*temp_0.5.txt"
file_paths = glob.glob(os.path.join(directory, file_pattern))

BG = read_files_into_dataframe(file_paths[0])
UDN = read_files_into_dataframe(file_paths[1])
DDD = read_files_into_dataframe(file_paths[2])

In [107]:
def calculate_mean_score(df):
    df["solvability_score"] = df["solvability_score"].astype(float)
    grouped_df = df.groupby("filename")["solvability_score"].mean().reset_index()
    # TODO something going wrong here in this function with mean
    return grouped_df

In [108]:
BG_mean = calculate_mean_score(BG)
UDN_mean = calculate_mean_score(UDN)
DDD_mean = calculate_mean_score(DDD)

In [112]:
def bin_solvability_score(df):
    # Define bins and labels
    bins = [0, 33, 66, 100]
    labels = ['hard', 'medium', 'easy']
    df['solvability_score'] = df['solvability_score'].astype(float)
    # Use pd.cut to bin the 'solvability_score' column
    df['solvability_score_binned'] = pd.cut(df['solvability_score'], bins=bins, labels=labels, include_lowest=True, right=False)
    
    # Drop rows with NaN values in 'solvability_score_binned' column
    df = df.dropna(subset=['solvability_score_binned'])
    
    return df


In [113]:
BG_binned = bin_solvability_score(BG_mean)
UDN_binned = bin_solvability_score(UDN_mean)
DDD_binned = bin_solvability_score(DDD_mean)

In [117]:
# write to csv
BG_binned.to_csv("BG_binned.csv", index=False)
UDN_binned.to_csv("UDN_binned.csv", index=False)
DDD_binned.to_csv("DDD_binned.csv", index=False)